<p align="left">
  <img src="https://raw.githubusercontent.com/python35/IINTS-SDK/main/img/iints_logo.png" width="160">
</p>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/07_Ablation_Supervisor.ipynb)

# Ablation: With vs. Without the Supervisor

**Goal:** compare identical runs with vs. without the supervisor.


In [1]:
from __future__ import annotations
from pathlib import Path
import os
import sys
import subprocess


def _find_repo_root() -> Path | None:
    for root in [Path.cwd(), *Path.cwd().parents]:
        if (root / "pyproject.toml").exists() and (root / "src").exists():
            return root
    return None

repo_root = _find_repo_root()
if repo_root is None:
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    if not in_colab:
        raise RuntimeError("Run this notebook inside the IINTS-SDK repo or on Colab.")

    if not Path("IINTS-SDK").exists():
        subprocess.check_call(["git", "clone", "https://github.com/python35/IINTS-SDK.git"])
    repo_root = Path("IINTS-SDK").resolve()

os.chdir(repo_root)
sys.path.insert(0, str(repo_root / "src"))
print("Repo root:", repo_root)


Repo root: /home/runner/work/IINTS-SDK/IINTS-SDK


In [2]:
from iints.validation import load_patient_config_by_name
from iints.core.simulator import Simulator
from iints.core.algorithms.fixed_basal_bolus import FixedBasalBolus
from iints.core.patient.models import PatientModel
from iints.core.supervisor import SafetyLevel
import pandas as pd

patient_config = load_patient_config_by_name("clinic_safe_baseline").model_dump()
patient_config.update({"glucose_decay_rate": 0.01, "basal_insulin_rate": 0.5, "initial_glucose": 150.0})

patient = PatientModel(**patient_config)
algorithm = FixedBasalBolus(settings={"fixed_basal_rate": 0.5, "carb_ratio": 10.0})

def run_with_supervisor():
    sim = Simulator(patient_model=patient, algorithm=algorithm, time_step=5, seed=3)
    return sim.run_batch(240)

# No-op supervisor for ablation
class NoopSupervisor:
    def evaluate_safety(self, current_glucose, proposed_insulin, current_time, current_iob=0.0):
        return {
            "approved_insulin": proposed_insulin,
            "safety_level": SafetyLevel.SAFE,
            "actions_taken": [],
            "original_insulin": proposed_insulin,
            "insulin_reduction": 0.0,
            "emergency_mode": False,
            "safety_decision": None,
            "safety_reason": "APPROVED",
            "safety_triggered": False,
        }

    def get_safety_report(self):
        return {"total_violations": 0, "bolus_interventions_count": 0}

    def reset(self):
        pass


def run_without_supervisor():
    sim = Simulator(patient_model=patient, algorithm=algorithm, time_step=5, seed=3)
    sim.supervisor = NoopSupervisor()
    return sim.run_batch(240)

results_sup, safety_sup = run_with_supervisor()
results_unsup, safety_unsup = run_without_supervisor()

pd.DataFrame({
    "with_supervisor": results_sup["glucose_actual_mgdl"].describe(),
    "no_supervisor": results_unsup["glucose_actual_mgdl"].describe(),
})


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


,with_supervisor,no_supervisor
count,29.000000,29.000000
mean,76.763937,76.505274
std,37.286728,37.606957
min,26.116967,24.294420
25%,45.166066,44.922378
50%,70.933716,70.933716
75%,104.389461,104.389461
max,150.000000,150.000000


### Recap
This ablation isolates the effect of the supervisor.
